In [2]:
import numpy as np
import pandas as pd
from scipy import stats

import selective_recruitment.data as sdata
import selective_recruitment.globals as gl
import selective_recruitment.region as sreg

import Functional_Fusion.dataset as ds
import Functional_Fusion.atlas_map as am

# statistically investigate conjunctions between the contrasts of interest at the coarse areal level
areas - multiple demand regions identified in hierarchical parcellations

In [3]:
dataset = "WMFS"
ses_id = "ses-02"
atlas_suit = "SUIT3"
parcellation = "NettekovenSym68c32"

In [4]:
# get cerebellar data
data_cereb,info,dset = ds.get_dataset(gl.base_dir,
                                dataset = dataset,
                                atlas=atlas_suit,
                                sess=ses_id,
                                subj=None,
                                type = "CondAll",  
                                smooth = None)
# aggregate data over parcels
atlas, ainfo = am.get_atlas(atlas_dir=gl.atlas_dir, atlas_str=atlas_suit)
# get label file
labels = gl.atlas_dir + f'/{ainfo["dir"]}/atl-{parcellation}_space-SUIT_dseg.nii'
label_vec, _ = atlas.get_parcel(labels)
parcel_data, parcel_labels = ds.agg_parcels(data_cereb , 
                                                atlas.label_vector, 
                                                fcn=np.nanmean)
# # get the summary dataframe for the selected region
# D = sdata.get_summary_roi(data_cereb, info, 
#                             atlas_space = "SUIT3",
#                             atlas_roi = "NettekovenSym68c32", 
#                             unite_struct = False,
#                             add_rest = True, 
#                             var = "Y")
parcel_data.shape

(16, 12, 32)

In [5]:
# get contrast of interest (define all your contrasts here)
c_enc = np.zeros(len(info.index))
c_enc[info.phase == 0] = 1/(sum(info.phase == 0))
c_enc[info.phase == 1] = -1/(sum(info.phase == 1))

c_load = np.zeros(len(info.index))
c_load[info.load == 6] = 1/(sum(info.load == 6))
c_load[info.load == 2] = -1/(sum(info.load == 2))

c_recall = np.zeros(len(info.index))
c_recall[info.recall == 0] = 1/(sum(info.recall == 0))
c_recall[info.recall == 1] = -1/(sum(info.recall == 1))

c_loadB = np.zeros(len(info.index))
c_loadB[(info.load == 6) & (info.recall == 0)] = 1/ sum((info.load == 6) & (info.recall == 0))
c_loadB[(info.load == 2) & (info.recall == 0)] = -1/ sum((info.load == 2) & (info.recall == 0))


c_loadF = np.zeros(len(info.index))
c_loadF[(info.load == 6) & (info.recall == 1)] = 1/ sum((info.load == 6) & (info.recall == 1))
c_loadF[(info.load == 2) & (info.recall == 1)] = -1/ sum((info.load == 2) & (info.recall == 1))

c_dir6 = np.zeros(len(info.index))
c_dir6[(info.load == 6) & (info.recall == 0)] = 1/ sum((info.load == 6) & (info.recall == 0))
c_dir6[(info.load == 6) & (info.recall == 1)] = 1/ sum((info.load == 6) & (info.recall == 1))

c_dir4 = np.zeros(len(info.index))
c_dir4[(info.load == 4) & (info.recall == 0)] = 1/ sum((info.load == 4) & (info.recall == 0))
c_dir4[(info.load == 4) & (info.recall == 1)] = 1/ sum((info.load == 4) & (info.recall == 1))

c_dir2 = np.zeros(len(info.index))
c_dir2[(info.load == 2) & (info.recall == 0)] = 1/ sum((info.load == 2) & (info.recall == 0))
c_dir2[(info.load == 2) & (info.recall == 1)] = 1/ sum((info.load == 2) & (info.recall == 1))

c_loadFenc = np.zeros(len(info.index))
c_loadFenc[(info.load == 6) & (info.recall == 1) & (info.phase == 0)] = 1/ sum((info.load == 6) & (info.recall == 1) & (info.phase == 0))
c_loadFenc[(info.load == 2) & (info.recall == 1) & (info.phase == 0)] = -1/ sum((info.load == 2) & (info.recall == 1) & (info.phase == 0))


c_loadFret = np.zeros(len(info.index))
c_loadFret[(info.load == 6) & (info.recall == 1) & (info.phase == 1)] = 1/ sum(((info.load == 6) & (info.recall == 1) & (info.phase == 1)))
c_loadFret[(info.load == 2) & (info.recall == 1) & (info.phase == 1)] = -1/ sum(((info.load == 2) & (info.recall == 1) & (info.phase == 1)))

c_dirEnc = np.zeros(len(info.index))
c_dirEnc[(info.recall == 0) & (info.phase == 0)] = 1/(sum((info.recall == 0) & (info.phase == 0)))
c_dirEnc[(info.recall == 1) & (info.phase == 0)] = -1/(sum((info.recall == 1) & (info.phase == 0)))


c_dirRet = np.zeros(len(info.index))
c_dirRet[(info.recall == 0) & (info.phase == 1)] = 1/(sum((info.recall == 0) & (info.phase == 1)))
c_dirRet[(info.recall == 1) & (info.phase == 1)] = -1/(sum((info.recall == 1) & (info.phase == 1)))



In [7]:
# focus on multiple demand regions
## first get a mask for the selected D regions
mask, idx, coarse_selected_ = sreg.get_parcel_single(parcellation = "NettekovenSym68c32", 
                                                atlas_space = "SUIT3",
                                                roi_exp = "D..")
print("selected regions")                                                
print(coarse_selected_)
print(idx)

selected regions
['D1L', 'D2L', 'D3L', 'D4L', 'D1R', 'D2R', 'D3R', 'D4R']
[ 7  8  9 10 23 24 25 26]


## during encoding, what are the regions that show both recall dir and load effect

In [8]:
# load contrast (only in forwards condition) during encoding
data_con = c_loadFenc @ parcel_data
data_Ds = data_con[:, idx-1]
# perform one sample t-test against 0, p < 0.05, bonferroni corrected for 8 regions
# we are testing each contrast across selected regions 
T_results= stats.ttest_1samp(data_Ds, np.zeros(len(coarse_selected_)))
print()
print(T_results.pvalue)
sig_mask_loadEnc = T_results.pvalue<0.05/len(coarse_selected_)
print()
print("regions showing significance:")
print(np.array(coarse_selected_)[sig_mask_loadEnc])



[0.00305001 0.00779815 0.00349361 0.02097574 0.00134467 0.00424577
 0.00046197 0.00172127]

regions showing significance:
['D1L' 'D3L' 'D1R' 'D2R' 'D3R' 'D4R']


In [9]:
# recall direction contrast during encoding
data_con = c_dirEnc @ parcel_data
data_Ds = data_con[:, idx-1]
# perform one sample t-test against 0, p < 0.05, bonferroni corrected for 8 regions
# we are testing each contrast across selected regions 
T_results= stats.ttest_1samp(data_Ds, np.zeros(len(coarse_selected_)))
print()
print(T_results.pvalue)
sig_mask_dirEnc = T_results.pvalue<0.05/len(coarse_selected_)
print()
print("regions showing significance:")
print(np.array(coarse_selected_)[sig_mask_dirEnc])



[0.01146759 0.02071865 0.0068843  0.01003335 0.00184193 0.00190636
 0.00886556 0.00468639]

regions showing significance:
['D1R' 'D2R' 'D4R']


In [10]:
print("conjunction:")
conj_mask_enc = sig_mask_dirEnc * sig_mask_loadEnc
print(np.array(coarse_selected_)[conj_mask_enc])


conjunction:
['D1R' 'D2R' 'D4R']


## during retirval, what are the regions that show both recall dir and load effect

In [11]:
# load contrast (only in forwards condition) during retrieval
data_con = c_loadFret @ parcel_data
data_Ds = data_con[:, idx-1]
# perform one sample t-test against 0, p < 0.05, bonferroni corrected for 8 regions
# we are testing each contrast across selected regions 
T_results= stats.ttest_1samp(data_Ds, np.zeros(len(coarse_selected_)))
print()
print(T_results.pvalue)
sig_mask_loadRet = T_results.pvalue<0.05/len(coarse_selected_)
print()
print("regions showing significance:")
print(np.array(coarse_selected_)[sig_mask_loadRet])


[0.00100597 0.19117024 0.00493109 0.15146188 0.00056882 0.46253509
 0.00138825 0.04285027]

regions showing significance:
['D1L' 'D3L' 'D1R' 'D3R']


In [12]:
# recall direction contrast during retrieval
data_con = c_dirRet @ parcel_data
data_Ds = data_con[:, idx-1]
# perform one sample t-test against 0, p < 0.05, bonferroni corrected for 8 regions
# we are testing each contrast across selected regions 
T_results= stats.ttest_1samp(data_Ds, np.zeros(len(coarse_selected_)))
print()
print(T_results.pvalue)
sig_mask_dirRet = T_results.pvalue<0.05/len(coarse_selected_)
print()
print("regions showing significance:")
print(np.array(coarse_selected_)[sig_mask_dirRet])



[0.00010889 0.05194702 0.00179106 0.0139936  0.00079554 0.0087608
 0.00110773 0.00048121]

regions showing significance:
['D1L' 'D3L' 'D1R' 'D3R' 'D4R']


In [13]:
print("conjunction:")
conj_mask_ret = sig_mask_dirRet * sig_mask_loadRet
print(np.array(coarse_selected_)[conj_mask_ret])

conjunction:
['D1L' 'D3L' 'D1R' 'D3R']


# statistically investigate conjunctions between the contrasts of interest at the fine areal level
areas - all the areas within hierarchical parcelation

In [14]:
# selecting all the regions
regions = sreg.get_parcel_names(parcellation = parcellation, atlas_space = atlas_suit)
fine_selected_ = regions[1:]

# what are the regions that show activations in both encoding and retrieval

In [16]:
# load contrast (only in forwards condition) during encoding
data_con = c_enc @ parcel_data

# perform one sample t-test against 0, p < 0.05, bonferroni corrected for 8 regions
# we are testing each contrast across selected regions 
T_results= stats.ttest_1samp(data_con, np.zeros(data_con.shape[1]))
print()
# print(T_results.pvalue)
sig_mask_encVret = T_results.pvalue<0.05/data_con.shape[1]
print()
print("regions showing significance:")
print(np.array(fine_selected_)[sig_mask_encVret])



regions showing significance:
['A2L' 'S2L' 'M3R' 'M4R' 'A1R' 'A2R' 'I1R' 'S2R']


## during encoding, what are the regions that show both recall dir and load effect

In [17]:
# load contrast (only in forwards condition) during encoding
data_con = c_loadFenc @ parcel_data

# perform one sample t-test against 0, p < 0.05, bonferroni corrected for 8 regions
# we are testing each contrast across selected regions 
T_results= stats.ttest_1samp(data_con, np.zeros(data_con.shape[1]))
print()
# print(T_results.pvalue)
sig_mask_loadEnc = T_results.pvalue<0.05/data_con.shape[1]
print()
print("regions showing significance:")
print(np.array(fine_selected_)[sig_mask_loadEnc])



regions showing significance:
['M4L' 'M2R' 'D1R' 'D3R']


In [18]:
# load contrast (only in forwards condition) during encoding
data_con = c_dirEnc @ parcel_data

# perform one sample t-test against 0, p < 0.05, bonferroni corrected for 8 regions
# we are testing each contrast across selected regions 
T_results= stats.ttest_1samp(data_con, np.zeros(data_con.shape[1]))
print()
# print(T_results.pvalue)
sig_mask_loadEnc = T_results.pvalue<0.05/data_con.shape[1]
print()
print("regions showing significance:")
print(np.array(fine_selected_)[sig_mask_loadEnc])



regions showing significance:
[]


## during retrieval, what are the regions that show both recall dir and load effect

In [19]:
# load contrast (only in forwards condition) during encoding
data_con = c_loadFret @ parcel_data

# perform one sample t-test against 0, p < 0.05, bonferroni corrected for 8 regions
# we are testing each contrast across selected regions 
T_results= stats.ttest_1samp(data_con, np.zeros(data_con.shape[1]))
print()
# print(T_results.pvalue)
sig_mask_loadRet= T_results.pvalue<0.05/data_con.shape[1]
print()
print("regions showing significance:")
print(np.array(fine_selected_)[sig_mask_loadRet])



regions showing significance:
['M1L' 'D1L' 'D1R' 'D3R']


In [20]:
# load contrast (only in forwards condition) during encoding
data_con = c_dirRet @ parcel_data

# perform one sample t-test against 0, p < 0.05, bonferroni corrected for 8 regions
# we are testing each contrast across selected regions 
T_results= stats.ttest_1samp(data_con, np.zeros(data_con.shape[1]))
print()
# print(T_results.pvalue)
sig_mask_dirRet = T_results.pvalue<0.05/data_con.shape[1]
print()
print("regions showing significance:")
print(np.array(fine_selected_)[sig_mask_dirRet])



regions showing significance:
['M1L' 'A1L' 'D1L' 'M1R' 'D1R' 'D3R' 'D4R']


In [21]:
print("conjunction:")
conj_mask_ret = sig_mask_dirRet * sig_mask_loadRet
print(np.array(fine_selected_)[conj_mask_ret])

conjunction:
['M1L' 'D1L' 'D1R' 'D3R']
